In [40]:
import psycopg2
import pandas as pd
from fpdf import FPDF
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

He creado unas tablas vacías con la misma estructura que las tablas para introducir las que nos interese:

In [ ]:
CREATE TABLE wallapop_interesante AS
SELECT * FROM datos_wallpop_limpios WHERE 1=0;


CREATE TABLE vinted_interesante AS
SELECT * FROM datos_vinted_filtrados WHERE 1=0;



CREATE TABLE cash_converters_interesante AS
SELECT * FROM datos_cash_converters WHERE 1=0;



CREATE TABLE mercado_libre_interesante AS
SELECT * FROM datos_mercado_libre WHERE 1=0;

Decido hacer la media de venta de mi producto (ps5), para saber que productos me pueden interesar comprar

In [36]:
conn = psycopg2.connect(
    host="localhost",
    database="proyecto_5",
    user="postgres",
    password="admin"
)
cursor = conn.cursor()

## Wallapop

In [24]:
conn.rollback()

In [37]:
cursor = conn.cursor()

# Definir la consulta para calcular la media
media_productos_wallapop = """
SELECT AVG(precio) AS media_precio FROM datos_wallpop_limpios;
"""

# Ejecutar la consulta y obtener el resultado de la media
cursor.execute(media_productos_wallapop)
media_precio = cursor.fetchone()[0]  # Obtener solo el valor de la media

print("Media de precio:", media_precio)

# Paso 3: Insertar en wallapop_interesante los productos que cumplen el criterio
consulta_insertar = """
INSERT INTO wallapop_interesante (wallpop_id, user_id, titulo, descripcion, categoria_id, precio, 
                                  imagenes, reservado, localizacion, golpe, reacondicionado, 
                                  url_producto, posibilidad_envio)
SELECT wallpop_id, user_id, titulo, descripcion, categoria_id, precio, 
       imagenes, reservado, localizacion, golpe, reacondicionado, 
       url_producto, posibilidad_envio
FROM datos_wallpop_limpios
WHERE precio < %s;
"""

# Ejecutar la consulta de inserción con la media como parámetro
cursor.execute(consulta_insertar, (media_precio,))      

# Confirmar los cambios en la base de datos
conn.commit()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("Productos copiados a wallapop_interesante con éxito.")


Media de precio: 319.6429411764705882
Productos copiados a wallapop_interesante con éxito.


In [49]:
def obtener_datos_wallapop_sql():
    """
    Conecta a la base de datos y extrae los datos filtrados de Wallapop.
    Retorna un DataFrame con los datos.
    """
    # Conectar a la base de datos
    conn = psycopg2.connect(
        dbname="proyecto_5",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    
    query = "SELECT url_producto FROM wallapop_interesante"
    df = pd.read_sql_query(query, conn)
    
    # Cerrar conexión
    conn.close()
    
    return df

def generar_pdf(df, output_path="../data/informes_interesantes"):
    """
    Genera un PDF a partir de un DataFrame y lo guarda en output_path.
    """
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    
    # Título
    pdf.cell(200, 10, txt="Datos de Wallapop - Productos PS5", ln=True, align='C')
    pdf.ln(10)  # Salto de línea
    
    # Agregar filas de datos
    for i, row in df.iterrows():
        pdf.cell(200, 10, txt=", ".join(str(value) for value in row.values), ln=True)
    
    # Guardar el PDF
    pdf.output(output_path)

def enviar_correo_con_adjunto(destinatario, asunto, cuerpo, archivo_adj):
    """
    Envía un correo electrónico con un archivo adjunto.
    """
    remitente = "mdlflucas@gmail.com"
    contraseña = "slak evdq fdek farx"
    
    mensaje = MIMEMultipart()
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    mensaje['Subject'] = asunto
    mensaje.attach(MIMEText(cuerpo, 'plain'))
    
    # Adjuntar archivo PDF
    with open(archivo_adj, "rb") as adjunto:
        mime_base = MIMEBase('application', 'octet-stream')
        mime_base.set_payload(adjunto.read())
        encoders.encode_base64(mime_base)
        mime_base.add_header('Content-Disposition', f'attachment; filename={os.path.basename(archivo_adj)}')
        mensaje.attach(mime_base)
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(remitente, contraseña)
            server.send_message(mensaje)
        print("Correo enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")



In [50]:
# Obtener datos desde la base de datos
df_datos_wallapop = obtener_datos_wallapop_sql()

C:\Users\almaz\AppData\Local\Temp\ipykernel_19092\2915255581.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [51]:
# Generar PDF a partir de los datos
archivo_pdf = "datos_wallapop_interesantes.pdf"
generar_pdf(df_datos_wallapop, archivo_pdf)

In [52]:
# Enviar el PDF por correo
enviar_correo_con_adjunto(
    destinatario="marioosunamartinez10@gmail.com",
    asunto="Productos de Wallapop - PS5",
    cuerpo="Adjunto encontrarás el informe de productos de PS5 en Wallapop.",
    archivo_adj=archivo_pdf
)

Correo enviado exitosamente.
